# TRPO in OpenSim environment

## Import Libararies (OpenSim, ChainerRL, etc)

In [ ]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import
from future import standard_library
standard_library.install_aliases()  # NOQA
import argparse
import logging
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style


from osim.env import ProstheticsEnv

import chainer
from chainer import functions as F
from chainerrl.agents.trpo import TRPO
from chainerrl.agents.acer import ACERSeparateModel
from chainerrl.optimizers.nonbias_weight_decay import NonbiasWeightDecay
from chainerrl.optimizers import rmsprop_async
from chainerrl.action_value import QuadraticActionValue #Q-function output for continuous action space
#from chainerrl.action_value import DiscreteActionValue
import chainerrl
from chainerrl import experiments
from chainerrl import explorers
from chainerrl import misc
from chainerrl import policy
from chainerrl import q_functions
from chainerrl import replay_buffer
from chainerrl import v_functions

style.use('ggplot')

## For all experiments we use the same random seed
    GPU =-1 if you use CPU 
    GPU = 1 if you use GPU

In [ ]:
seed=0
gpu=-1

## Setting Hyper-parameters, Numbers of episodes and timesteps

In [ ]:
number_of_episodes=5000
max_episode_length=1000 # number of timesteps per episode

replay_start_size=5000
number_of_update_times=1

target_update_interval=1
target_update_method='soft'

soft_update_tau=1e-2
update_interval=4
number_of_eval_runs=100
eval_interval=10 ** 5

final_exploration_steps=10 ** 6

gamma=0.995
minibatch_size=128

## Defining reward, environment, random actions functions

In [ ]:
# Helper's functions

def clip_action_filter(a):
    return np.clip(a, action_space.low, action_space.high)

def reward_filter(r):
    return r 


def phi(obs):
    obs=np.array(obs)
    return obs.astype(np.float32)

def random_action():
    a = action_space.sample()
    if isinstance(a, np.ndarray):
        a = a.astype(np.float32)
    return a


def make_env(test,render=False):
    env = ProstheticsEnv(visualize=render)
    # Use different random seeds for train and test envs
    env_seed = 2 ** 32 - 1 - seed if test else seed
    env.seed(env_seed)
    #if args.monitor:
        #env = gym.wrappers.Monitor(env, args.outdir)
    if isinstance(env.action_space, spaces.Box):
        misc.env_modifiers.make_action_filtered(env, clip_action_filter)
    if not test:
        misc.env_modifiers.make_reward_filtered(env, reward_filter)
    if render and not test:
        misc.env_modifiers.make_rendered(env)
    return env

In [ ]:
# Set a random seed used in ChainerRL
misc.set_random_seed(seed)

In [ ]:
env = make_env(test=False,render=False) # render=True for visualisation 
obs_size = np.asarray(env.observation_space.shape).prod()
action_space = env.action_space

action_size = np.asarray(action_space.shape).prod()

## Defining Value function, policy function and optimizer function adam 

In [ ]:
obs_space = env.observation_space
# Value function 
v_fuc = chainerrl.v_functions.FCVFunction(
        obs_space.low.size,
        n_hidden_channels=64,
        n_hidden_layers=4,
        last_wscale=0.01,
        nonlinearity=F.tanh,
)


v_fuc_opt = chainer.optimizers.Adam()
v_fuc_opt.setup(v_fuc)

# Policy function 
pi =  chainerrl.policies.FCGaussianPolicyWithStateIndependentCovariance(
                obs_space.low.size,
                action_space.low.size,
                n_hidden_channels=64,
                n_hidden_layers=3,
                mean_wscale=0.01,
                nonlinearity=F.tanh,
                var_type='diagonal',
                var_func=lambda x: F.exp(2 * x),  # Parameterize log std
                var_param_init=0,  # log std = 0 => std = 1
)

In [ ]:
rbuf = replay_buffer.ReplayBuffer(5 * 10)

ou_sigma = (action_space.high - action_space.low) * 0.2

explorer = explorers.AdditiveOU(sigma=ou_sigma)
#obs_normalizer = chainerrl.links.MLPConvolution2D(obs_space.low.size)
obs_normalizer = chainerrl.links.EmpiricalNormalization(obs_space.low.size)

## Agent (algorithm) function 

In [ ]:
# The agent
agent = TRPO(
            policy=pi,
                 vf=v_fuc,
                 vf_optimizer = v_fuc_opt,
                 obs_normalizer=obs_normalizer,
                phi=lambda x: np.array(x).astype(np.float32, copy=False),
                update_interval=update_interval,
                conjugate_gradient_max_iter=100,
                conjugate_gradient_damping=1e-3,
                gamma=gamma,
                lambd=0.97,
                vf_epochs=5,
                entropy_coef=0,
            )
#agent.load("TRPO_Prosthetic_model_edited_parameters_anther 2000_V2")

## Training, save reward in Text file and plot results

In [ ]:
G=[]
G_mean=[]
for ep in range(1, number_of_episodes+ 1):
    if ep%100:
        agent.save("TRPO_Prosthetic_5000")
    obs = env.reset()
    reward = 0
    done = False
    R = 0  # return (sum of rewards)
    t = 0  # time step
    episode_rewards=[]
    while not done and t < max_episode_length:
        # Uncomment to watch the behaviour
        env.render()
        action = agent.act_and_train(obs, reward)
        obs, reward, done, _ = env.step(action)
        R += reward
        episode_rewards.append(reward)
        t += 1
        
    if done or t >= max_episode_length :
            
            # Calculate sum of the rewards
        episode_rewards_sum = sum(episode_rewards)     
        G.append(episode_rewards_sum)
        total_G = np.sum(G)
        maximumReturn = np.amax(G)
        print("%f" % (episode_rewards_sum), file=open("TRPO_Prosthetic_5000.txt", "a"))
        #print("%i" % (episode_rewards_sum))
        if ep % 10 == 0:
                
            print("==========================================")
            print("Episode: ", ep)
            print("Rewards: ", episode_rewards_sum)
            print("Max reward so far: ", maximumReturn)
            # Mean reward
            total_reward_mean = np.divide(total_G, ep+1)
            G_mean.append(total_reward_mean)
            print("Mean Reward", total_reward_mean)
            print("%f" % (total_reward_mean), file=open("TRPO_MEAN_Prosthetic_5000.txt", "a"))    
                
    agent.stop_episode_and_train(obs, reward, done)
    
    
print('Finished.')

plt.xlabel('episdes')
plt.ylabel('reword')
plt.plot(G)   
plt.savefig('trpo_prosthetic_5000', dpi = 1000)


plt.plot(G_mean)
plt.ylabel('Average of Returns')
plt.xlabel('Number of episodes/10')

plt.savefig("ReturnsAverage_VS_Episodes trpo_prosthetic_5000", dpi = 1000)